***GENERATED CODE FOR distractionindicator PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Timestamp', 'transformation_label': 'String Indexer'}], 'feature': 'Timestamp', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '481', 'mean': '', 'stddev': '', 'min': '11/12/2022 13:16', 'max': '7/14/2022 19:33', 'missing': '0', 'distinct': '444'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Timestamp'}, {'feature_label': 'Timestamp', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Timestamp')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '481', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'unsure ', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Relationship Status', 'transformation_label': 'String Indexer'}], 'feature': 'Relationship Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '481', 'mean': '', 'stddev': '', 'min': 'Divorced', 'max': 'Single', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Relationship Status'}, {'feature_label': 'Relationship Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Relationship Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Occupation Status', 'transformation_label': 'String Indexer'}], 'feature': 'Occupation Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '481', 'mean': '', 'stddev': '', 'min': 'Retired', 'max': 'University Student', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Occupation Status'}, {'feature_label': 'Occupation Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Occupation Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Organizations Affiliated', 'transformation_label': 'String Indexer'}], 'feature': 'Organizations Affiliated', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '481', 'mean': '', 'stddev': '', 'min': 'Company', 'max': 'University, Private', 'missing': '0', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Organizations Affiliated'}, {'feature_label': 'Organizations Affiliated', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Organizations Affiliated')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Social Media Usage', 'transformation_label': 'String Indexer'}], 'feature': 'Social Media Usage', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '481', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Social Media Usage'}, {'feature_label': 'Social Media Usage', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Social Media Usage')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Social Media Platforms', 'transformation_label': 'String Indexer'}], 'feature': 'Social Media Platforms', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '481', 'mean': '', 'stddev': '', 'min': 'Discord', 'max': 'YouTube, Snapchat, Discord, Reddit', 'missing': '0', 'distinct': '125'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Social Media Platforms'}, {'feature_label': 'Social Media Platforms', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Social Media Platforms')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Average Time on Social Media', 'transformation_label': 'String Indexer'}], 'feature': 'Average Time on Social Media', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '481', 'mean': '', 'stddev': '', 'min': 'Between 1 and 2 hours', 'max': 'More than 5 hours', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Average Time on Social Me...'}, {'feature_label': 'Average Time on Social Media', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Average Time on Social Media')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run distractionindicatorHooks.ipynb
try:
	#sourcePreExecutionHook()

	modifiedsocialmediaandmentalhealth = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/ModifiedSocialMediaAndMentalHealth.csv', 'filename': 'ModifiedSocialMediaAndMentalHealth.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(modifiedsocialmediaandmentalhealth)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run distractionindicatorHooks.ipynb
try:
	#transformationPreExecutionHook()

	distractionindicatorautofe = TransformationMain.run(modifiedsocialmediaandmentalhealth,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Timestamp", "transformation_label": "String Indexer"}], "feature": "Timestamp", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "481", "mean": "", "stddev": "", "min": "11/12/2022 13:16", "max": "7/14/2022 19:33", "missing": "0", "distinct": "444"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Timestamp"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "481", "mean": "26.14", "stddev": "9.92", "min": "13.0", "max": "91.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "481", "mean": "", "stddev": "", "min": "Female", "max": "unsure ", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "Relationship Status", "transformation_label": "String Indexer"}], "feature": "Relationship Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "481", "mean": "", "stddev": "", "min": "Divorced", "max": "Single", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Relationship Status"}, {"transformationsData": [{"feature_label": "Occupation Status", "transformation_label": "String Indexer"}], "feature": "Occupation Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "481", "mean": "", "stddev": "", "min": "Retired", "max": "University Student", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Occupation Status"}, {"transformationsData": [{"feature_label": "Organizations Affiliated", "transformation_label": "String Indexer"}], "feature": "Organizations Affiliated", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "481", "mean": "", "stddev": "", "min": "Company", "max": "University, Private", "missing": "0", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Organizations Affiliated"}, {"transformationsData": [{"feature_label": "Social Media Usage", "transformation_label": "String Indexer"}], "feature": "Social Media Usage", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "481", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Social Media Usage"}, {"transformationsData": [{"feature_label": "Social Media Platforms", "transformation_label": "String Indexer"}], "feature": "Social Media Platforms", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "481", "mean": "", "stddev": "", "min": "Discord", "max": "YouTube, Snapchat, Discord, Reddit", "missing": "0", "distinct": "125"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Social Media Platforms"}, {"transformationsData": [{"feature_label": "Average Time on Social Media", "transformation_label": "String Indexer"}], "feature": "Average Time on Social Media", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "481", "mean": "", "stddev": "", "min": "Between 1 and 2 hours", "max": "More than 5 hours", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Average Time on Social Me..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Frequency of using Social media without a specific purpose", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "3.55", "stddev": "1.1", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Frequency of using Social..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Distraction Frequency", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "3.32", "stddev": "1.33", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Distraction Frequency"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Restlessness Level", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "2.59", "stddev": "1.26", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Restlessness Level"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Distractibility Level", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "3.35", "stddev": "1.18", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Distractibility Level"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Frequency of bothered by worries", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "3.56", "stddev": "1.28", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Frequency of bothered by ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Difficulty to concentrate", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "3.25", "stddev": "1.35", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Difficulty to concentrate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Comparison Frequency", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "2.83", "stddev": "1.41", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Comparison Frequency"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": " Feelings about Comparisons", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "2.78", "stddev": "1.06", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": " Feelings about Compariso..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Validation Seeking Frequency on social media?", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "2.46", "stddev": "1.25", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Validation Seeking Freque..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Depression Frequency", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "3.26", "stddev": "1.31", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Depression Frequency"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Daily Activity Fluctuation Level", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "3.17", "stddev": "1.26", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Daily Activity Fluctuatio..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sleep Issues Frequency", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "3.2", "stddev": "1.46", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Sleep Issues Frequency"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Distraction Indicator", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "481", "mean": "0.7", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Distraction Indicator"}]}))

	#transformationPostExecutionHook(distractionindicatorautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run distractionindicatorHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(distractionindicatorautofe, ["Age", "Frequency of using Social media without a specific purpose", "Distraction Frequency", "Restlessness Level", "Distractibility Level", "Frequency of bothered by worries", "Difficulty to concentrate", "Comparison Frequency", " Feelings about Comparisons", "Validation Seeking Frequency on social media?", "Depression Frequency", "Daily Activity Fluctuation Level", "Sleep Issues Frequency", "Timestamp_stringindexer", "Gender_stringindexer", "Relationship Status_stringindexer", "Occupation Status_stringindexer", "Organizations Affiliated_stringindexer", "Social Media Usage_stringindexer", "Social Media Platforms_stringindexer", "Average Time on Social Media_stringindexer"], "Distraction Indicator")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

